# Lab 9 - DEFI(Uniswap V3)

## 2. Deploy UniswapV3 Contracts Locally

In [ ]:
artifacts = {
    WETH9: require("./build/WETH9.json"),
    UniswapV3Factory: require("./build/UniswapV3Factory.json"),
    UniswapV3Pool: require('./build/UniswapV3Pool.json'),
    NFTDescriptor: require("./build/NFTDescriptor.json"),
    NonfungibleTokenPositionDescriptor: require("./build/NonfungibleTokenPositionDescriptor.json"),
    NonfungiblePositionManager: require("./build/NonfungiblePositionManager.json"),
};

In [ ]:
new Promise(async ()=>{

    const { ethers } = require("ethers");
    const provider = new ethers.providers.JsonRpcProvider("http://127.0.0.1:8545");
    const account = provider.getSigner();

    // Deploy WETH
    WETH9 = new ethers.ContractFactory(
        artifacts.WETH9.abi,
        artifacts.WETH9.bytecode,
        account
    );
    weth = await WETH9.deploy();

    // Deploy UniswapV3Factory
    UniswapV3Factory = new ethers.ContractFactory(
        artifacts.UniswapV3Factory.abi,
        artifacts.UniswapV3Factory.bytecode,
        account
    );
    factory = await UniswapV3Factory.deploy();

    // Deploy NFTDescriptor
    NFTDescriptor = new ethers.ContractFactory(
        artifacts.NFTDescriptor.abi,
        artifacts.NFTDescriptor.bytecode,
        account
    );
    nftDescriptor = await NFTDescriptor.deploy();

    // Create Linked Bytecode
    const { utils } = require("ethers");

    const linkLibraries = ({ bytecode, linkReferences }, libraries) => {
        Object.keys(linkReferences).forEach((fileName) => {
            Object.keys(linkReferences[fileName]).forEach((contractName) => {
                if (!libraries.hasOwnProperty(contractName)) {
                    throw new Error(
                        `Missing link library name ${contractName}`
                    );
                }
                const address = utils
                    .getAddress(libraries[contractName])
                    .toLowerCase()
                    .slice(2);
                linkReferences[fileName][contractName].forEach(
                    ({ start, length }) => {
                        const start2 = 2 + start * 2;
                        const length2 = length * 2;
                        bytecode = bytecode
                            .slice(0, start2)
                            .concat(address)
                            .concat(
                                bytecode.slice(
                                    start2 + length2,
                                    bytecode.length
                                )
                            );
                    }
                );
            });
        });
        return bytecode;
    };

    const linkedBytecode = linkLibraries(
        {
            bytecode: artifacts.NonfungibleTokenPositionDescriptor.bytecode,
            linkReferences: {
                "NFTDescriptor.sol": {
                    NFTDescriptor: [
                        {
                            length: 20,
                            start: 1681, // old value is 1261
                        },
                    ],
                },
            },
        },
        {
            NFTDescriptor: nftDescriptor.address,
        }
    );

    const NonfungibleTokenPositionDescriptor = new ethers.ContractFactory(
        artifacts.NonfungibleTokenPositionDescriptor.abi,
        linkedBytecode,
        account
    );

    const label = ethers.utils.formatBytes32String("WETH");
    nonfungibleTokenPositionDescriptor =
        await NonfungibleTokenPositionDescriptor.deploy(weth.address, label);    

    const NonfungiblePositionManager = new ethers.ContractFactory(
        artifacts.NonfungiblePositionManager.abi,
        artifacts.NonfungiblePositionManager.bytecode,
        account
    );

    nonfungiblePositionManager = await NonfungiblePositionManager.deploy(
        factory.address,
        weth.address,
        nonfungibleTokenPositionDescriptor.address
    );

    contracts = {
        WETH: weth.address,
        UNISWAPV3_FACTORY: factory.address,
        NONFUNGIBLE_POSITION_MANAGER: nonfungiblePositionManager.address,
    };
    console.log(contracts);

})